# M3.4 - Creating a Basin-Scale Water Budget

*Part of:* [**Open Science for Water Resources**](https://github.com/OpenClimateScience/M3-Open-Science-for-Water-Resources)

In [ ]:
import glob
import datetime
import earthaccess
import numpy as np
import h5py
import xarray as xr
from matplotlib import pyplot

auth = earthaccess.login()

$$
P = E + R + \Delta S
$$

![](./assets/water_budget.png)

[*Image courtesy of the USGS*](https://www.usgs.gov/media/images/components-a-simple-water-budget-part-a-watershed)

--- 

## Estimating evapotranspiration for our basin

In [ ]:
import geopandas

basin = geopandas.read_file('/home/arthur.endsley/Workspace/NTSG/projects/Y2024_TOPS_Training/data/YellowstoneRiver_drainage_WSG84.shp')

In [ ]:
basin.bounds.to_numpy().ravel()

In [ ]:
tuple(basin.bounds.to_numpy().ravel())

In [ ]:
# https://dx.doi.org/10.5067/MODIS/MOD16A3GF.061

results = earthaccess.search_data(
    short_name = 'MOD16A3GF',
    temporal = ('2014-01-01', '2018-12-31'),
    bounding_box = tuple(basin.bounds.to_numpy().ravel()))

In [ ]:
results_clean = list(filter(lambda granule: 'h10v04' in granule['meta']['native-id'], results))

In [ ]:
earthaccess.download(results_clean, 'data/MOD16A3')

### Installing a new Python package

In [ ]:
!pip install --upgrade py4eos

In [ ]:
import py4eos

hdf = py4eos.read_hdf4eos('data/MOD16A3/MOD16A3GF.A2014001.h10v04.061.2022077145846.hdf')
hdf

In [ ]:
et = hdf.get('ET_500m')
et

In [ ]:
et = hdf.get('ET_500m', scale_and_offset = True)
et

In [ ]:
ds_et = hdf.to_rasterio('ET_500m', filename = '', driver = 'MEM')
ds_et

### Compiling an ET time series for our basin

In [ ]:
file_list = glob.glob('data/MOD16A3/*.hdf')
file_list

In [ ]:
gtiff_file_list = []

for filename in file_list:
    year = filename.split('.')[1].replace('A', '').replace('001', '')
    hdf = py4eos.read_hdf4eos(filename)
    output_filename = f'processed/MOD16A3GF_{year}_ET_500m.tiff'
    ds_et = hdf.to_rasterio(
        'ET_500m', filename = output_filename, scale_and_offset = True)
    ds_et.close()
    gtiff_file_list.append(output_filename)

In [ ]:
import rioxarray

ds_et = rio.open(gtiff_file_list[0])
ds_et

In [ ]:
transform = ds_et.transform
transform

In [ ]:
ul_x, ul_y = tuple(transform)[2], tuple(transform)[5]
(ul_x, ul_y)

In [ ]:
ul_point = geopandas.points_from_xy(x = [ul_x], y = [ul_y], crs = ds_et.crs).to_crs(epsg = 5070)
ul_point

In [ ]:
transform = list(transform.to_gdal())
transform

In [ ]:
transform[0] = ul_point.x[0]
transform[3] = ul_point.y[0]
transform

In [ ]:
transform

In [ ]:
from affine import Affine
from rasterio.warp import reproject, Resampling

et_series = []

for filename in gtiff_file_list:
    ds_et = rio.open(filename)
    ds_et_albers = rio.open('', 'w+', driver = 'MEM', count = 1,
        dtype = np.float32, width = ds_et.width, height = ds_et.height, 
        crs = basin_albers.crs, transform = Affine.from_gdal(*transform))
    reproject(
        source = rio.band(ds_et, 1),
        destination = rio.band(ds_et_albers, 1), 
        resampling = Resampling.bilinear)
    break


---

## Obtaining basin-scale runoff data

In [ ]:
import xarray as xr

ds = xr.open_dataset('/home/arthur/Workspace/NTSG/projects/Y2024_TOPS_Training/data/HYSETS_watershed_YellowstoneRiver.nc')
ds

In [ ]:
ds_4years = ds.sel(time = slice('2014-01-01', '2018-12-31'))
ds_4years

In [ ]:
ds_4years['discharge'].plot()

In [ ]:
# Compute total daily discharge based on the mean rate
ds_4years['discharge_total'] = ds_4years.discharge * 60 * 60 * 24

In [ ]:
# https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#dateoffset-objects

# Compute annual discharge
ds_4years['discharge_total'].resample(time = 'YS').sum().to_series().plot.bar()

## Computing basin-scale runoff

### Calculating basin area

In [ ]:
basin.crs

https://epsg.io/5070

In [ ]:
basin.to_crs(epsg = 5070).crs

In [ ]:
basin_area = basin.to_crs(epsg = 5070).area.values

# Convert from [m] to [km]
basin_area / 1e6

In [ ]:
1.789e11 / 1e6

In [ ]:
# Get the [m year-1] that this basin drained through Yellowstone River,
#  then convert from [m year-1] to [mm year-1]
runoff_meters_per_yr = ds_4years['discharge_total'].resample(time = 'YS').sum() / basin_area
runoff_mm_per_yr = runoff_meters_per_yr * 1000
runoff_mm_per_yr